In [1]:
import pandas as pd
import numpy as np

In [ ]:
План:
    генерация датасета
    показыать, что в цсв не сохраняются типы и долгое сохранение\открытие, индекс=Фолс
    сравнение цсв, пикл, паркета
    открытие больших файлов:
        - как использовать цикл
        - явное указание типов, индексов, дат
        - как использовать батчи
    ускорение вычислений (сослаться в ноуте про красивости на то, что ускорение будет показано в уроке про оптмизицию)

## Использование Numpy для ускорения вычислений

### операции внутри массива

In [2]:
%%timeit

a = list(range(1_000_000))
[e * e for e in a]

91.6 ms ± 1.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
%%timeit

a = np.arange(1_000_000)
a * a

2.63 ms ± 41.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### аггрегирующие функции

In [5]:
b = np.random.randint(0, 10, size=(1000, 1000))
b[36, 42] = 20
b = b.ravel()

In [6]:
%%timeit

max(b)

39.7 ms ± 375 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%%timeit

b.max()

80.6 µs ± 2.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Если все таки хочется использовать списки, то лучше не используйте явные циклы

In [9]:
# добавить в список несколько элементов
a = ['a', 'b', 'c']
b = ['d', 'e', 'f']
print(a, b)
a.extend(b)
print(a)

['a', 'b', 'c'] ['d', 'e', 'f']
['a', 'b', 'c', 'd', 'e', 'f']


In [20]:
# проверить наличия элемента в списке или во множестве. Что быстрее?
a = list(range(1_000)) * 1_000_000

In [21]:
%%timeit

999 in a

9.68 µs ± 62 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [22]:
a = set(list(range(1_000)) * 1_000_000)

In [24]:
%%timeit

999 in a

45.6 ns ± 0.282 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### использование генераторов

In [25]:
L = list(range(1_000_000))

In [26]:
%%timeit

res = []
for i in L:
    res.append(i + 10)

94.1 ms ± 808 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
%%timeit

res = [i + 10 for i in L]

65.5 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### ускорение в 2 раза, но можно посмотреть на numpy

In [30]:
%%timeit

res = np.array(L) + 10

54.4 ms ± 285 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
# не сильно быстрее из-за конвертации списка в массив numpy. Но если сделать один раз ее, а потом работать с массивом
L_array = np.array(L)

In [32]:
%%timeit

res = L_array + 10

1.17 ms ± 61.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### если не хватает памяти на чтение всех файлов, их можно обрбабатывать генератором

In [33]:
def gen(path_to_file):
#     with open(path_to_file, 'r', encoding='utf-8') as fin:
#         line = fin.readline()
    yield 1#, line

## Использование pandas

In [2]:
df = pd.read_csv('titanic.csv')
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.info() #смотрим memory usage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## Явное преобразование типов для уменьшения использования памяти

In [6]:
df[['PassengerId', 'Survived', 'Pclass', 'SibSp', 'Parch']] = \
    df[['PassengerId', 'Survived', 'Pclass', 'SibSp', 'Parch']].astype(np.int8)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int8   
 1   Survived     891 non-null    int8   
 2   Pclass       891 non-null    int8   
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int8   
 7   Parch        891 non-null    int8   
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int8(5), object(5)
memory usage: 53.2+ KB


преобразуем типы и посмотрим что будет с используемой памятью

In [7]:
df[['Age', 'Fare']] = df[['Age', 'Fare']].astype(np.float32)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int8   
 1   Survived     891 non-null    int8   
 2   Pclass       891 non-null    int8   
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float32
 6   SibSp        891 non-null    int8   
 7   Parch        891 non-null    int8   
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float32
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float32(2), int8(5), object(5)
memory usage: 46.2+ KB


In [8]:
df[['Sex', 'Embarked']] = df[['Sex', 'Embarked']].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int8    
 1   Survived     891 non-null    int8    
 2   Pclass       891 non-null    int8    
 3   Name         891 non-null    object  
 4   Sex          891 non-null    category
 5   Age          714 non-null    float32 
 6   SibSp        891 non-null    int8    
 7   Parch        891 non-null    int8    
 8   Ticket       891 non-null    object  
 9   Fare         891 non-null    float32 
 10  Cabin        204 non-null    object  
 11  Embarked     889 non-null    category
dtypes: category(2), float32(2), int8(5), object(3)
memory usage: 34.3+ KB


Также можно при чтении указать типа колонки. делается это параметром dtype = {'Age': np.float32}

## Сохранение промежуточных этапов

In [48]:
df = pd.read_csv('transactions_train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


In [49]:
%%time
# хорошо работает при больших данных

df.to_parquet('train.parquet', index=False)
df = pd.read_parquet('train.parquet')

CPU times: total: 20.6 s
Wall time: 22.3 s


In [50]:
%%time
# универсальный способ

df.to_csv('train.csv', index=False)
df = pd.read_csv('train.csv')

CPU times: total: 2min 22s
Wall time: 3min 32s


In [51]:
%%time
# быстрый способ при нужных форматах данных int/bool/float

df.to_pickle('train.pkl')
df = pd.read_pickle('train.pkl')

CPU times: total: 13.5 s
Wall time: 21.1 s


## Многопоточное apply

In [ ]:
%pip install swifter

In [58]:
import swifter

In [59]:
%%time

tmp = df['price'].apply(lambda x: x**2)

CPU times: total: 8.89 s
Wall time: 10.5 s


In [60]:
%%time

tmp = df['price'].swifter.apply(lambda x: x**2)

Dask Apply:   0%|          | 0/12 [00:00<?, ?it/s]

CPU times: total: 3.98 s
Wall time: 16.4 s


#### инициализация занимает немного времени, но процесс идет намного быстрее

### gropby

In [63]:
%%time

tmp = df.groupby('article_id')['price'].sum()

CPU times: total: 1.27 s
Wall time: 1.43 s


In [64]:
%%time
## но здесь тип колонки для groupby должен быть int

tmp = np.bincount(df['article_id'], weights=df['price'])

CPU times: total: 297 ms
Wall time: 341 ms


### самое часто встречаемое значение

In [65]:
df['article_id'].value_counts()

706016001    50287
706016002    35043
372860001    31718
610776002    30199
759871002    26329
             ...  
520736002        1
619777003        1
586904003        1
512385003        1
533261032        1
Name: article_id, Length: 104547, dtype: int64

### если данных очень много и они не помещаются в оперативную память, то используем dask

In [66]:
import dask.dataframe as dd

In [73]:
ddf = dd.read_csv('transactions_train.csv')

In [75]:
ddf.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


#### особенность dask: чтение и фильтрация данных не начнется, пока не запуститься команда .compute()

зато можно посмотреть типы данных, head, задать фильтры и только потом зачитать целиком файл, делать select по колонкам 

этот принцип называется lazy_evaluation

In [76]:
ddf = ddf[ddf.sales_channel_id==2]

In [77]:
ddf.compute()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
...,...,...,...,...,...
588584,2020-09-22,ffd4cf2217de4a0a3f9f610cdec334c803692a18af08ac...,902288001,0.022017,2
588585,2020-09-22,ffd4cf2217de4a0a3f9f610cdec334c803692a18af08ac...,856440002,0.042356,2
588586,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
588587,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2


https://docs.dask.org/en/stable/

### Так же вычисления данных типа int или float можно проводить на GPU

https://rapids.ai/start.html

https://github.com/rapidsai/cudf

## Еще пару советов:

1. Используйте явную типизацию колонок в датасете
2. Мультипроцессинг хорошо работает в MacOs и Linux, Windows не всегда в него умеет. Советую поставить wsl2
3. Для числовых данных используйте numpy(под капотом у него C++), вместо списков